## 1. 라이브러리 설치

In [ ]:

!pip install timm albumentations -q

## 2. Import Libraries

In [ ]:
import os
import time
import random
import warnings
warnings.filterwarnings('ignore')

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.utils.class_weight import compute_class_weight


## 3. 시드 고정

In [ ]:
# 시드 고정
SEED = 42

os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## 4. Hyperparameters (EDA 결과 반영)

In [ ]:
# ========== 하이퍼파라미터 (EDA 결과 반영) ========== #
CFG = {
    # 데이터
    'IMG_SIZE': 224,           # EDA: 평균 484x551 → 224로 resize
    'BATCH_SIZE': 32,
    'NUM_CLASSES': 17,
    
    # 학습
    'EPOCHS': 20,              # 더 긴 학습
    'LR': 1e-4,
    'WEIGHT_DECAY': 1e-5,      # 정규화 추가
    
    # 모델
    'MODEL_NAME': 'resnet50',  # 'efficientnet_b3', 'convnext_base'
    
    # K-Fold
    'N_FOLDS': 1,
    'FOLD_TO_RUN': 0,
    
    # 경로 (EDA 파일과 동일)
    'BASE_DIR': '/home/realtheai/cv_competetion',
    'TRAIN_CSV': '/home/realtheai/cv_competetion/data/train.csv',
    'SAMPLE_SUBMISSION_CSV': '/home/realtheai/cv_competetion/data/sample_submission.csv',
    'TRAIN_IMG_DIR': '/home/realtheai/cv_competetion/data/train',
    'TEST_IMG_DIR': '/home/realtheai/cv_competetion/data/test',
    
    # 디바이스
    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu',
    
    # Augmentation 강도 (EDA 결과: Test에 많은 노이즈)
    'AUG_STRENGTH': 0.7,       # 0.5 → 0.7 (강화!)
    
    # 클래스 가중치 사용 여부 (EDA 결과: 2.2:1 불균형)
    'USE_CLASS_WEIGHTS': True,
}

print("\n=== Configuration (EDA 결과 반영) ===")
for k, v in CFG.items():
    print(f"{k:25s}: {v}")

## 5. Dataset 클래스

In [5]:
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        
        # RGB로 변환 (일부 grayscale 이미지 대응)
        img = np.array(Image.open(img_path).convert('RGB'))
        
        if self.transform:
            img = self.transform(image=img)['image']
        
        return img, target

## 6. Augmentation 정의 (EDA 결과 반영)

In [ ]:
def get_train_transforms(img_size=224, aug_strength=0.7):

    return A.Compose([
        # 크기 조정
        A.Resize(img_size, img_size),
        
        # ========== EDA 발견: 기울어짐 대응 ========== #
        A.Perspective(scale=(0.05, 0.1), p=aug_strength * 0.5),  # 원근 왜곡
        A.Rotate(limit=15, p=aug_strength),                      # 회전 (10→15도)
        A.ShiftScaleRotate(
            shift_limit=0.1, 
            scale_limit=0.15, 
            rotate_limit=15, 
            p=aug_strength
        ),
        
        # ========== EDA 발견: 구겨짐 대응 ========== #
        A.GridDistortion(num_steps=5, distort_limit=0.3, p=aug_strength * 0.5),
        A.OpticalDistortion(distort_limit=0.3, shift_limit=0.3, p=aug_strength * 0.3),
        
        # ========== EDA 발견: 밝기 차이 대응 (중요!) ========== #
        A.RandomBrightnessContrast(
            brightness_limit=0.3,   # 0.2 → 0.3 강화!
            contrast_limit=0.3,     # 0.2 → 0.3 강화!
            p=aug_strength
        ),
        A.RandomGamma(gamma_limit=(70, 130), p=aug_strength),  # (80,120) → (70,130)
        
        # ========== EDA 발견: 빛 반사/번짐 대응 ========== #
        A.RandomShadow(
            shadow_roi=(0, 0.5, 1, 1), 
            num_shadows_limit=(1, 3),  # 1-2 → 1-3 증가
            p=aug_strength * 0.5
        ),
        A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), p=aug_strength * 0.2),  # 빛번짐 추가
        
        # ========== 노이즈 추가 ========== #
        A.OneOf([
            A.GaussNoise(var_limit=(10, 50)),
            A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5)),
        ], p=aug_strength * 0.5),
        
        A.OneOf([
            A.GaussianBlur(blur_limit=(3, 7)),      # (3,5) → (3,7)
            A.MotionBlur(blur_limit=(3, 7)),
        ], p=aug_strength * 0.5),
        
        # ========== EDA 발견: 가림 대응 ========== #
        A.CoarseDropout(
            max_holes=12,          # 8 → 12 증가
            max_height=32, 
            max_width=32, 
            p=aug_strength * 0.5
        ),
        
        # 색상 변화
        A.HueSaturationValue(
            hue_shift_limit=10, 
            sat_shift_limit=20, 
            val_shift_limit=10, 
            p=aug_strength * 0.3
        ),
        
        # 정규화 및 텐서 변환
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

def get_valid_transforms(img_size=224):
    """Validation용 transform (augmentation 없음)"""
    return A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])


## 7. 모델 생성

In [ ]:
def create_model(model_name='resnet50', num_classes=17, pretrained=True):
    """timm을 사용한 모델 생성"""
    model = timm.create_model(
        model_name,
        pretrained=pretrained,
        num_classes=num_classes
    )
    return model

# 모델 생성 테스트
model = create_model(
    model_name=CFG['MODEL_NAME'],
    num_classes=CFG['NUM_CLASSES']
)
model = model.to(CFG['DEVICE'])

print(f"\n✅ 모델 생성 완료: {CFG['MODEL_NAME']}")
print(f"   - 총 파라미터: {sum(p.numel() for p in model.parameters()):,}")
print(f"   - 학습 가능 파라미터: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

## 8. 학습/검증 함수

In [8]:
def train_one_epoch(loader, model, optimizer, loss_fn, device, scheduler=None):
    """1 epoch 학습"""
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader, desc='Training')
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()
        
        if scheduler is not None:
            scheduler.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())
        
        pbar.set_postfix({'loss': loss.item()})

    avg_loss = train_loss / len(loader)
    acc = accuracy_score(targets_list, preds_list)
    f1 = f1_score(targets_list, preds_list, average='macro')
    
    return avg_loss, acc, f1


def validate(loader, model, loss_fn, device):
    """검증"""
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Validation')
        for image, targets in pbar:
            image = image.to(device)
            targets = targets.to(device)
            
            preds = model(image)
            loss = loss_fn(preds, targets)
            
            val_loss += loss.item()
            preds_list.extend(preds.argmax(1).cpu().numpy())
            targets_list.extend(targets.cpu().numpy())
            
            pbar.set_postfix({'loss': loss.item()})
    
    avg_loss = val_loss / len(loader)
    acc = accuracy_score(targets_list, preds_list)
    f1 = f1_score(targets_list, preds_list, average='macro')
    
    return avg_loss, acc, f1, preds_list, targets_list

print("✅ 학습/검증 함수 정의 완료!")

✅ 학습/검증 함수 정의 완료!


## 9. 틀린 이미지 분석 함수 

In [ ]:
def analyze_wrong_predictions(val_df, preds_list, targets_list, img_dir, meta_df=None, save_path=None):

    # 틀린 인덱스 찾기
    wrong_indices = [i for i, (pred, target) in enumerate(zip(preds_list, targets_list)) if pred != target]
    
    print(f"\n{'='*60}")
    print(f"📊 검증 결과 분석")
    print(f"{'='*60}")
    print(f"   전체: {len(preds_list)}개")
    print(f"   정답: {len(preds_list) - len(wrong_indices)}개")
    print(f"   오답: {len(wrong_indices)}개 ({len(wrong_indices)/len(preds_list)*100:.1f}%)")
    print(f"   정확도: {(len(preds_list) - len(wrong_indices))/len(preds_list)*100:.1f}%")
    
    if len(wrong_indices) == 0:
        print("\n🎉 모든 예측이 정확합니다!")
        return
    
    # 클래스별 오류 분석
    wrong_by_true_class = {}
    for idx in wrong_indices:
        true_class = targets_list[idx]
        pred_class = preds_list[idx]
        if true_class not in wrong_by_true_class:
            wrong_by_true_class[true_class] = []
        wrong_by_true_class[true_class].append((idx, pred_class))
    
    print(f"\n📈 클래스별 오답 분석:")
    for class_id in sorted(wrong_by_true_class.keys()):
        count = len(wrong_by_true_class[class_id])
        if meta_df is not None:
            class_name = meta_df[meta_df['target'] == class_id]['class_name'].values[0]
            print(f"   Class {class_id:2d} ({class_name[:20]:20s}): {count:2d}개 오답")
        else:
            print(f"   Class {class_id:2d}: {count:2d}개 오답")
    
    # 틀린 이미지 최대 12개 시각화
    if save_path:
        n_show = min(12, len(wrong_indices))
        fig, axes = plt.subplots(3, 4, figsize=(16, 12))
        axes = axes.flatten()
        
        for i, idx in enumerate(wrong_indices[:n_show]):
            img_name = val_df.iloc[idx]['ID']
            img_path = os.path.join(img_dir, img_name)
            
            if os.path.exists(img_path):
                img = Image.open(img_path)
                axes[i].imshow(img)
                
                true_label = f"True: {targets_list[idx]}"
                pred_label = f"Pred: {preds_list[idx]}"
                
                if meta_df is not None:
                    true_class = meta_df[meta_df['target'] == targets_list[idx]]['class_name'].values[0]
                    pred_class = meta_df[meta_df['target'] == preds_list[idx]]['class_name'].values[0]
                    true_label += f"\n({true_class[:15]})"
                    pred_label += f"\n({pred_class[:15]})"
                
                axes[i].set_title(
                    f"{true_label}\n{pred_label}",
                    fontsize=9,
                    color='red',
                    fontweight='bold'
                )
                axes[i].axis('off')
        
        # 빈 subplot 숨기기
        for i in range(n_show, 12):
            axes[i].axis('off')
        
        plt.tight_layout()
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.show()
        print(f"\n💾 틀린 이미지 저장: {save_path}")
    

print("✅ 결과 분석 함수 정의 완료!")

## 10. 데이터 로드 및 전처리

In [ ]:
# 데이터 로드
train_df = pd.read_csv(CFG['TRAIN_CSV'])
test_df = pd.read_csv(CFG['SAMPLE_SUBMISSION_CSV'])
meta_df = pd.read_csv(f"{CFG['BASE_DIR']}/data/meta.csv")

print(f"✅ 데이터 로드 완료")
print(f"   - Train: {len(train_df)}개")
print(f"   - Test: {len(test_df)}개")
print(f"   - 클래스: {len(meta_df)}개")

# Transform 생성
train_transform = get_train_transforms(
    img_size=CFG['IMG_SIZE'],
    aug_strength=CFG['AUG_STRENGTH']
)
valid_transform = get_valid_transforms(img_size=CFG['IMG_SIZE'])

print(f"\n✅ Transform 생성 완료")
print(f"   - Augmentation 강도: {CFG['AUG_STRENGTH']}")

## 11. 클래스 가중치 계산 (EDA 발견: 2.2:1 불균형)

In [ ]:
# 클래스 분포 확인
class_counts = train_df['target'].value_counts().sort_index()
print("\n📊 클래스 분포:")
for class_id, count in class_counts.items():
    class_name = meta_df[meta_df['target'] == class_id]['class_name'].values[0]
    print(f"   Class {class_id:2d} ({class_name[:25]:25s}): {count:3d}개")

# 불균형 비율 계산
max_count = class_counts.max()
min_count = class_counts.min()
imbalance_ratio = max_count / min_count

print(f"\n⚠️ 클래스 불균형:")
print(f"   최대: {max_count}개 / 최소: {min_count}개")
print(f"   비율: {imbalance_ratio:.2f}:1")

# 클래스 가중치 계산
if CFG['USE_CLASS_WEIGHTS']:
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(train_df['target']),
        y=train_df['target']
    )
    class_weights = torch.FloatTensor(class_weights).to(CFG['DEVICE'])
    
    print(f"\n✅ 클래스 가중치 적용:")
    for i, weight in enumerate(class_weights):
        print(f"   Class {i:2d}: {weight:.4f}")
else:
    class_weights = None
    print(f"\n❌ 클래스 가중치 미사용")

## 12. Train/Validation 분할

In [ ]:
# Train/Validation 분할 (80:20, Stratified)
train_idx, val_idx = train_test_split(
    range(len(train_df)),
    test_size=0.2,
    random_state=SEED,
    stratify=train_df['target']
)

# CSV 파일 임시 생성
train_df.iloc[train_idx].to_csv(f"{CFG['BASE_DIR']}/train_fold.csv", index=False)
train_df.iloc[val_idx].to_csv(f"{CFG['BASE_DIR']}/val_fold.csv", index=False)

# Dataset 생성
trn_dataset = ImageDataset(f"{CFG['BASE_DIR']}/train_fold.csv", CFG['TRAIN_IMG_DIR'], train_transform)
val_dataset = ImageDataset(f"{CFG['BASE_DIR']}/val_fold.csv", CFG['TRAIN_IMG_DIR'], valid_transform)

# DataLoader 생성
trn_loader = DataLoader(
    trn_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

print(f"✅ DataLoader 생성 완료")
print(f"   - Train: {len(trn_loader)} batches ({len(trn_dataset)}개)")
print(f"   - Valid: {len(val_loader)} batches ({len(val_dataset)}개)")

## 13. 학습 준비

In [ ]:
# 모델, 옵티마이저, 손실 함수, 스케줄러
model = create_model(CFG['MODEL_NAME'], CFG['NUM_CLASSES']).to(CFG['DEVICE'])

optimizer = Adam(
    model.parameters(), 
    lr=CFG['LR'],
    weight_decay=CFG['WEIGHT_DECAY']
)

# Weighted Loss (EDA 결과 반영)
if CFG['USE_CLASS_WEIGHTS']:
    loss_fn = nn.CrossEntropyLoss(weight=class_weights)
    print("✅ Weighted CrossEntropyLoss 사용")
else:
    loss_fn = nn.CrossEntropyLoss()
    print("✅ CrossEntropyLoss 사용")

# Cosine Annealing Scheduler
scheduler = CosineAnnealingLR(
    optimizer, 
    T_max=CFG['EPOCHS'] * len(trn_loader),
    eta_min=1e-6
)

print(f"\n✅ 학습 준비 완료")
print(f"   - Optimizer: Adam (lr={CFG['LR']}, wd={CFG['WEIGHT_DECAY']})")
print(f"   - Scheduler: CosineAnnealingLR")
print(f"   - Loss: {'Weighted ' if CFG['USE_CLASS_WEIGHTS'] else ''}CrossEntropyLoss")

## 14. 학습 실행 

In [ ]:
print("\n" + "="*80)
print("학습 시작!")
print("="*80)

best_f1 = 0
best_epoch = 0
history = {
    'train_loss': [], 'train_acc': [], 'train_f1': [],
    'val_loss': [], 'val_acc': [], 'val_f1': []
}

for epoch in range(CFG['EPOCHS']):
    print(f"\n{'='*80}")
    print(f"Epoch {epoch+1}/{CFG['EPOCHS']}")
    print(f"{'='*80}")
    
    # Train
    train_loss, train_acc, train_f1 = train_one_epoch(
        trn_loader, model, optimizer, loss_fn, CFG['DEVICE'], scheduler
    )
    
    # Validation
    val_loss, val_acc, val_f1, preds, targets = validate(
        val_loader, model, loss_fn, CFG['DEVICE']
    )
    
    # History 저장
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['train_f1'].append(train_f1)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)
    
    # 결과 출력
    print(f"\n📊 Epoch {epoch+1} Results:")
    print(f"   Train - Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | F1: {train_f1:.4f}")
    print(f"   Valid - Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | F1: {val_f1:.4f}")
    
    # Best 모델 저장
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_epoch = epoch + 1
        torch.save(model.state_dict(), f"{CFG['BASE_DIR']}/best_model.pth")
        print(f"   🎉 Best model saved! (F1: {best_f1:.4f})")
    
    # Early Stopping 체크 (간단 버전)
    if epoch - best_epoch >= 5:
        print(f"\n⚠️ Early Stopping! (Best epoch: {best_epoch})")
        break

print(f"\n{'='*80}")
print(f"🎉 학습 완료!")
print(f"   Best Validation F1: {best_f1:.4f} (Epoch {best_epoch})")
print(f"{'='*80}")

## 15. 학습 결과 시각화

In [ ]:
# 학습 곡선 그리기
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train', marker='o')
axes[0].plot(history['val_loss'], label='Valid', marker='o')
axes[0].set_title('Loss', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Accuracy
axes[1].plot(history['train_acc'], label='Train', marker='o')
axes[1].plot(history['val_acc'], label='Valid', marker='o')
axes[1].set_title('Accuracy', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(alpha=0.3)

# F1 Score
axes[2].plot(history['train_f1'], label='Train', marker='o')
axes[2].plot(history['val_f1'], label='Valid', marker='o')
axes[2].axhline(y=best_f1, color='r', linestyle='--', label=f'Best: {best_f1:.4f}')
axes[2].set_title('Macro F1 Score', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('F1 Score')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f"{CFG['BASE_DIR']}/training_curves.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"💾 학습 곡선 저장: {CFG['BASE_DIR']}/training_curves.png")

## 16. 틀린 이미지 분석

In [ ]:
# Best 모델 로드
model.load_state_dict(torch.load(f"{CFG['BASE_DIR']}/best_model.pth"))
model.eval()

# 최종 검증
val_loss, val_acc, val_f1, preds, targets = validate(
    val_loader, model, loss_fn, CFG['DEVICE']
)

# 틀린 이미지 분석
val_df = pd.read_csv(f"{CFG['BASE_DIR']}/val_fold.csv")

analyze_wrong_predictions(
    val_df=val_df,
    preds_list=preds,
    targets_list=targets,
    img_dir=CFG['TRAIN_IMG_DIR'],
    meta_df=meta_df,
    save_path=f"{CFG['BASE_DIR']}/wrong_predictions.png"
)

## 17. Confusion Matrix

In [ ]:
# Confusion Matrix 시각화
cm = confusion_matrix(targets, preds)

plt.figure(figsize=(14, 12))
plt.imshow(cm, cmap='Blues', interpolation='nearest')
plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
plt.colorbar()

# 축 레이블 (클래스 이름)
tick_labels = [f"{i}\n{meta_df[meta_df['target']==i]['class_name'].values[0][:10]}" 
               for i in range(CFG['NUM_CLASSES'])]
plt.xticks(range(CFG['NUM_CLASSES']), tick_labels, rotation=45, ha='right', fontsize=8)
plt.yticks(range(CFG['NUM_CLASSES']), tick_labels, fontsize=8)

plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)

# 각 셀에 숫자 표시
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, str(cm[i, j]), 
                ha='center', va='center',
                color='white' if cm[i, j] > cm.max()/2 else 'black',
                fontsize=8)

plt.tight_layout()
plt.savefig(f"{CFG['BASE_DIR']}/confusion_matrix.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💾 Confusion Matrix 저장: {CFG['BASE_DIR']}/confusion_matrix.png")
print("\n💡 Confusion Matrix 해석:")
print("   - 대각선: 정확하게 예측한 개수")
print("   - 대각선 외: 잘못 예측한 개수")

## 18. Test 예측 및 제출 파일 생성

In [ ]:
# Best 모델 로드 (이미 로드되어 있지만 재확인)
model.load_state_dict(torch.load(f"{CFG['BASE_DIR']}/best_model.pth"))
model.eval()

# Test Dataset
test_df.to_csv(f"{CFG['BASE_DIR']}/test_temp.csv", index=False)
tst_dataset = ImageDataset(f"{CFG['BASE_DIR']}/test_temp.csv", CFG['TEST_IMG_DIR'], valid_transform)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

# 예측
print("\n🔮 Test 데이터 예측 중...")
preds_list = []
with torch.no_grad():
    for image, _ in tqdm(tst_loader, desc='Predicting'):
        image = image.to(CFG['DEVICE'])
        preds = model(image)
        preds_list.extend(preds.argmax(1).cpu().numpy())

# 제출 파일 생성
pred_df = pd.DataFrame({
    'ID': test_df['ID'],
    'target': preds_list
})

submission_path = f"{CFG['BASE_DIR']}/submission.csv"
pred_df.to_csv(submission_path, index=False)

print(f"\n✅ 제출 파일 생성 완료: {submission_path}")
print(f"\n📊 예측 결과 미리보기:")
print(pred_df.head(10))

# 예측 분포 확인
print(f"\n📈 예측 클래스 분포:")
pred_counts = pd.Series(preds_list).value_counts().sort_index()
for class_id, count in pred_counts.items():
    class_name = meta_df[meta_df['target'] == class_id]['class_name'].values[0]
    print(f"   Class {class_id:2d} ({class_name[:25]:25s}): {count:4d}개")